<h1 align='center'>Preprocesamiento de datos y análisis inicial</h1>
<h2>Déjame escuchar la música</h2>

Como un analista de datos, el trabajo consiste en analizar datos para extraer información valiosa y tomar decisiones basadas en ellos. Esto implica diferentes etapas, como la descripción general de los datos, el preprocesamiento y la prueba de hipótesis.

Siempre que investigamos, necesitamos formular hipótesis que después podamos probar. A veces aceptamos estas hipótesis; y otras veces, las rechazamos. Para tomar las decisiones correctas, una empresa debe ser capaz de entender si está haciendo las suposiciones correctas.

En este proyecto, se comparará las preferencias musicales de las ciudades de Springfield y Shelbyville. Se estudiarán los datos reales de transmisión de música online para probar las hipótesis y se comparará el comportamiento de los usuarios de estas dos ciudades.

### Objetivo:
Probar dos hipótesis:
1. La actividad de los usuarios y las usuarias difiere según el día de la semana y dependiendo de la ciudad.
2. Los oyentes de Springfield y Shelbyville tienen preferencias distintas. En Springfield prefieren el pop, mientras que en Shelbyville hay más personas a las que les gusta el rap.

### Etapas
Los datos del comportamiento del usuario se almacenan en el archivo `/datasets/music_project_en.csv`. No hay ninguna información sobre la calidad de los datos, así que se necesitará examinarlos antes de probar las hipótesis.

Primero, se evaluará la calidad de los datos y se verá si los problemas son significativos. Entonces, durante el preprocesamiento de datos, se tomará en cuenta los problemas más críticos.

*El proyecto consistirá en tres etapas:*
 1. Descripción de los datos
 2. Preprocesamiento de datos
 3. Prueba de hipótesis

### Diccionario de datos

Los registros de canciones contienen los siguientes campos:
- `' userID'` — identificador del usuario;
- `'Track'` — título de la canción;
- `'artist'` — nombre del artista;
- `'genre'` — género musical;
- `'City'` — ciudad del usuario;
- `'time'` — hora exacta en la que se reprodujo la canción;
- `'Day'` — día de la semana.

# Contenido <a id='back'></a>

* [Etapa 1. Descripción de los datos](#data_review)
    * [Conclusiones](#data_review_conclusions)
* [Etapa 2. Preprocesamiento de datos](#data_preprocessing)
    * [2.1 Estilo del encabezado](#header_style)
    * [2.2 Valores ausentes](#missing_values)
    * [2.3 Duplicados](#duplicates)
    * [2.4 Conclusiones](#data_preprocessing_conclusions)
* [Etapa 3. Prueba de hipótesis](#hypotheses)
    * [3.1 Hipótesis 1: actividad de los usuarios y las usuarias en las dos ciudades](#activity)
    * [3.2 Hipótesis 2: preferencias de género en Springfield y Shelbyville](#genre)
* [Conclusiones](#end)

## Etapa 1. Descripción de los datos <a id='data_review'></a>

In [1]:
import pandas as pd

In [2]:
# Acceso a los registros del recurso "music_project_en.csv"
df = pd.read_csv('./datasets/music_project_en.csv')

# Obtener información general sobre los registros del DataFrame

display(df.head(10))

df.info()

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


### Observaciones <a id='data_review_conclusions'></a>
Dadas las observaciones iniciales, se pueden identificar problemas con los encabezados de la tabla, ya que no presentan uniformidad en la nomenclatura: algunos están en mayúsculas, otros en minúsculas y se evidencian espacios iniciales y finales en los nombres de algunos encabezados. Por ello, se optará por emplear la notación `snake_case` en los encabezados. Las columnas del DataFrame son de tipo `object`, almacenando cadenas de caracteres, es decir, datos de tipo texto. Los datos proporcionados son adecuados para realizar un análisis y comprobación de hipótesis. Existen datos ausentes en los campos de `genre`, `artist` y `track`.


[Volver a Contenidos](#back)

## Etapa 2. Preprocesamiento de datos <a id='data_preprocessing'></a>

El objetivo aquí es preparar los datos para que sean analizados.

El primer paso es resolver cualquier problema con los encabezados, luego podemos avanzar a los valores ausentes y duplicados.


### Estilo del encabezado <a id='header_style'></a>

In [3]:
# Encabezados del DataFrame
print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')



Reglas de la nomenclatura `snake_case`:
* Todos los caracteres deben estar en minúsculas.
* Se deben eliminar todos los espacios iniciales y finales.
* Si el nombre tiene varias palabras, los espacios entre palabras se reemplazarán con un guión bajo.


In [4]:

# Aplicar notación snake_case a los encabezados del DataFrame
new_col_names = []

for old_name in df.columns:
    new_col_names.append(old_name.strip().lower().replace(' ', '_'))

# Reemplaza los nombres anteriores por los nuevos
df.columns = new_col_names

# Cambiar el nombre del encabezado "user_id"
df.rename(columns={'userid':'user_id'}, inplace=True)

df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

[Volver a Contenidos](#back)

### Valores ausentes <a id='missing_values'></a>

In [5]:
print("Valores ausentes en las columnas del DataFrame: ")
df.isna().sum()

Valores ausentes en las columnas del DataFrame: 


user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64

No todos los valores ausentes afectan la investigación. Por ejemplo, los valores ausentes en `track` y `artist` no son cruciales y pueden ser reemplazados con valores predeterminados como `'unknown'` (desconocido).

Sin embargo, los valores ausentes en `genre` pueden afectar la comparación entre las preferencias musicales de Springfield y Shelbyville. En la vida real, sería útil conocer las razones de los datos ausentes e intentar recuperarlos, pero no se tiene esa oportunidad en este proyecto. Por lo tanto, se rellenarán estos valores ausentes con un valor predeterminado y se evaluará cuánto podrían afectar estos valores ausentes a los cálculos.

In [6]:
# Reemplazar los valores ausentes de las columnas ['track','artist','genre'] con 'unknown'

columns_to_replace = ['track','artist','genre']

for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

display(df.sample(10))
print("Valores ausentes en las columnas del DataFrame: ")
df.isna().sum()

,user_id,track,artist,genre,city,time,day
43819,9C91380,Chan Chan,Buena Vista Social Club,world,Springfield,13:26:39,Friday
7181,868AFB02,I Pray (FYH081),Frainbreeze,dance,Springfield,09:09:41,Friday
26710,F4BA42FF,Aint Safe,Doitmovin,hip,Springfield,09:57:44,Wednesday
14511,56DF9A3D,Tornado,PPK,electronic,Springfield,13:37:03,Monday
49327,EDA8FA39,Idunn,Trobar De Morte,folk,Shelbyville,13:28:43,Wednesday
13009,F941397F,Pinger,Bad District,electronic,Springfield,21:25:16,Wednesday
39886,C894BDFF,Johnny Go!,NikitA,ruspop,Springfield,14:56:19,Wednesday
41348,DD071E82,With U,FRSH KEPT,dance,Springfield,08:51:33,Monday
50589,1C45B685,Key Of Life,Ohmna,dance,Shelbyville,13:42:25,Friday
22142,E509EEE2,Into You,unknown,dance,Springfield,20:24:46,Friday


Valores ausentes en las columnas del DataFrame: 


user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

[Volver a Contenidos](#back)

### Duplicados <a id='duplicates'></a>

In [7]:
print(f"Existen {df.duplicated().sum()} valores duplicados en el DataFrame")

Existen 3826 valores duplicados en el DataFrame


In [8]:
# Eliminación de duplicados explícitos

df = df.drop_duplicates().reset_index(drop=True) 

print(f"Existen {df.duplicated().sum()} valores duplicados en el DataFrame")

Existen 0 valores duplicados en el DataFrame


Ahora queremos eliminar los duplicados implícitos en la columna `genre`. Por ejemplo, el nombre de un género puede escribirse de varias formas, y dichos errores también pueden afectar el resultado. Para hacerlo, primero mostraremos una lista de nombres de género únicos, ordenados alfabéticamente.


In [9]:
# Inspeccionando los nombres de géneros únicos

df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'ambient', 'americana', 'animated', 'anime',
       'arabesk', 'arabic', 'arena', 'argentinetango', 'art', 'audiobook',
       'avantgarde', 'axé', 'baile', 'balkan', 'beats', 'bigroom',
       'black', 'bluegrass', 'blues', 'bollywood', 'bossa', 'brazilian',
       'breakbeat', 'breaks', 'broadway', 'cantautori', 'cantopop',
       'canzone', 'caribbean', 'caucasian', 'celtic', 'chamber',
       'children', 'chill', 'chinese', 'choral', 'christian', 'christmas',
       'classical', 'classicmetal', 'club', 'colombian', 'comedy',
       'conjazz', 'contemporary', 'country', 'cuban', 'dance',
       'dancehall', 'dancepop', 'dark', 'death', 'deep', 'deutschrock',
       'deutschspr', 'dirty', 'disco', 'dnb', 'documentary', 'downbeat',
       'downtempo', 'drum', 'dub', 'dubstep', 'eastern', 'easy',
       'electronic', 'electropop', 'emo', 'entehno', 'epicmetal',
       'estrada', 'ethnic', 'eurofo

Busca en la lista para encontrar duplicados implícitos del género `hiphop`. Estos pueden ser nombres escritos incorrectamente o nombres alternativos para el mismo género.

Verás los siguientes duplicados implícitos:
* `hip`
* `hop`
* `hip-hop`

Para eliminarlos, se declara la función `replace_wrong_values()` con dos parámetros:
* `wrong_genres` — la lista de duplicados.
* `correct_genre` — la cadena con el valor correcto.

La función debería corregir los nombres en la columna `'genre'` del DataFrame, reemplazando cada valor de la lista `wrong_genres` con el valor en `correct_genre`.

In [10]:
# Reemplazar duplicados implícitos
def replace_wrong_values(df, column, wrong_values, correct_value):
    '''
        Args: 
            df (DataFrame): Data frame al cual se va a realizar los cambios
            column (string): Columna a la cual se va a reemplazar el duplicado implícito
            wrong_values (list): Lista de duplicados,
            correct_value (string): Valor correcto
    '''
    for wrong_value in wrong_values:
        df[column] = df[column].replace(wrong_value, correct_value)
    

In [11]:
replace_wrong_values(df, 'genre', ['hip', 'hop', 'hip-hop'], 'hiphop')

[Volver a Contenidos](#back)

### Observaciones <a id='data_preprocessing_conclusions'></a>

Aunque un analista de datos pueda observar o comprender que ciertas variaciones en un campo pueden referirse a un mismo caso específico, como es el caso de los duplicados implícitos de `hiphop`, donde `hip`, `hop` y `hip-hop` son esencialmente lo mismo, el programa los trataría como valores distintos si no se manejan adecuadamente. Esto podría resultar en datos incorrectos. Por esta razón, es crucial saber cómo acceder a los valores de una columna e identificar duplicados que podrían ser reemplazados por un único valor, para garantizar la precisión en los análisis y resultados.

[Volver a Contenidos](#back)

## Etapa 3. Prueba de hipótesis <a id='hypotheses'></a>

### Hipótesis 1: Comparar el comportamiento del usuario en las dos ciudades <a id='activity'></a>

La primera hipótesis afirma que existen diferencias en la forma en que los usuarios de Springfield y Shelbyville consumen música. Para comprobar esto, se usarán los datos de tres días de la semana: lunes, miércoles y viernes.

In [12]:
# Contando las canciones reproducidas en cada ciudad

df.groupby(by='city')['track'].count()

city
Shelbyville    18512
Springfield    42741
Name: track, dtype: int64

**Observaciones**

*Comentario:* Springfield tiene significativamente más canciones reproducidas (45,360) en comparación con Shelbyville (19,719). Esto sugiere que la actividad musical en Springfield es más alta durante este período de tres días.

*Posibles razones:* La diferencia en la cantidad de canciones reproducidas puede deberse a varios factores, como la población de la ciudad, disponibilidad de servicios de transmisión de música, entre otros.

In [13]:
# Calculando las canciones reproducidas en cada uno de los tres días

df.groupby(by='day')['track'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

**Observaciones**

*Comentario:* El número de canciones reproducidas varía a lo largo de la semana, siendo viernes el día con la mayor cantidad de reproducciones, seguido por el lunes y luego el miércoles.

*Inferencia:* Dado que el viernes es un día asociado como el comienzo de los días de descanso, el fin de semana, sugiere que los usuarios tienden a ser más activos musicalmente. Esto podría deberse a que las personas tienden a relajarse y disfrutar de la música. Igualmente los lunes suelen tener un mayor número de reproducciones musicales y podría ser un indicativo de que la gente busca la música para comenzar bien la semana.


La función `number_tracks()` calcula el número de canciones reproducidas en un determinado día y ciudad. La función debe aceptar dos parámetros:

- `day`: un día de la semana para filtrar. Por ejemplo, `'Monday'`.
- `city`: una ciudad para filtrar. Por ejemplo, `'Springfield'`.

Dentro de la función, se aplicará un filtrado consecutivo con indexación lógica.

Después de filtrar los datos por dos criterios, se cuenta el número de valores de la columna 'user_id' en la tabla resultante. Este recuento representa el número de entradas que se está buscando. 

In [14]:
# La función cuenta las pistas reproducidas en un cierto día y ciudad.

def number_tracks(day, city):
    '''
        Args: 
            `day`: un día de la semana para filtrar. Por ejemplo, `'Monday'`.
            `city`: una ciudad para filtrar. Por ejemplo, `'Springfield'`.
    '''
    track_list = df[(df['day']==day) & (df['city']==city)]
    track_list_count = track_list['user_id'].count()

    return track_list_count

Se crea una nueva tabla, donde:
* Los encabezados de la tabla son: `['city', 'monday', 'wednesday', 'friday']`
* Los datos son los resultados que se consiguieron de `number_tracks()`

In [15]:
# Tabla con los resultados
data_number_tracks = [
    [
        'Springfield', 
         number_tracks('Monday', 'Springfield'), 
         number_tracks('Wednesday', 'Springfield'), 
         number_tracks('Friday', 'Springfield')],
    [
        'Shelbyville', 
         number_tracks('Monday', 'Shelbyville'), 
         number_tracks('Wednesday', 'Shelbyville'), 
         number_tracks('Friday', 'Shelbyville')
    ]
]
df_number_tracks = pd.DataFrame(data=data_number_tracks, columns=['city', 'monday', 'wednesday', 'friday'])
print(df_number_tracks)

          city  monday  wednesday  friday
0  Springfield   15740      11056   15945
1  Shelbyville    5614       7003    5895


**Conclusiones**

*Hipótesis:* La actividad de los usuarios difiere según el día de la semana y dependiendo de la ciudad.

*Razonamiento:* Hay variaciones en la actividad de reproducción entre los días de la semana para ambas ciudades. Por ejemplo, en Springfield, el viernes tiene una actividad significativamente mayor en comparación con el miércoles, mientras que en Shelbyville, el miércoles es el día con la actividad más alta. En general, Springfield tiene números más altos de reproducción en comparación con Shelbyville en todos los días de la semana. Por lo tanto se acepta la primera hipótesis.

[Volver a Contenidos](#back)

### Hipótesis 2: preferencias de género en Springfield y Shelbyville <a id='genre'></a>

Hipótesis: Shelbyville ama la música rap. A los residentes de Springfield les gusta más el pop.

In [16]:
# Top géneros en Springfield

spr_general = df[df['city']=='Springfield']

spr_genres = spr_general.groupby(by='genre')['genre'].count().sort_values(ascending=False)

spr_genres[:10]

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

In [17]:
# Top géneros en Shelbyville

shel_general = df[df['city']=='Shelbyville']

shel_genres = shel_general.groupby(by='genre')['genre'].count().sort_values(ascending=False)

shel_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Conclusión**

*Hipótesis:* Los oyentes de Springfield y Shelbyville tienen preferencias distintas. En Springfield prefieren el pop, mientras que en Shelbyville hay más personas a las que les gusta el rap.

*Razonamiento:* Las preferencias de géneros musicales en las ciudades de Shelbyville y Springfield son similares, dada la recolección de datos se tiene que los géneros más escuhcados son: pop, dance y rock. Por lo tanto se rechaza la segunda hipótesis. 

[Volver a Contenidos](#back)

# Conclusiones <a id='end'></a>

**Conclusiones sobre cada hipótesis:**

1. *Hipótesis 1:* La actividad de los usuarios difiere según el día de la semana y dependiendo de la ciudad.
   - *Resultado:* La actividad de reproducción varía entre los días de la semana para ambas ciudades, con Springfield mostrando una actividad generalmente más alta. La hipótesis se acepta debido a las diferencias notables en la actividad entre las dos ciudades y en diferentes días de la semana.

2. *Hipótesis 2:* Los oyentes de Springfield y Shelbyville tienen preferencias distintas. En Springfield prefieren el pop, mientras que en Shelbyville hay más personas a las que les gusta el rap.
   - *Resultado:* Las preferencias de géneros musicales en las ciudades de Shelbyville y Springfield son similares, con los géneros más escuchados siendo pop, dance y rock en ambas ciudades. No hay evidencia que respalde la afirmación de preferencias distintas. La hipótesis se rechaza debido a la falta de diferencias significativas en las preferencias de géneros.

En resumen, las conclusiones indican que, en general, hay diferencias en la actividad musical entre ciudades y días de la semana (Hipótesis 1), pero no se encontraron diferencias sustanciales en los géneros musicales preferidos en el experimento de Hipótesis 3. Estas observaciones ayudan a comprender el comportamiento de los usuarios en sus preferencias musicales.

[Volver a Contenidos](#back)